In [23]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config


from classification.models.M5 import M5, M5PLModule

from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

# Train Model

In [3]:
from classification.models.M5 import M5, M5PLModule

hparams = {
    "batch_size": 64,
    "learning_rate": 3e-4,
    "weight_decay": 0.001,
    "lr_decay": 0.95
}

model = M5PLModule(hparams)
model.prepare_data()

#
#

Loading cached train data from /nfs/students/summer-term-2020/project-4/data/data_8k
Loading cached val data from /nfs/students/summer-term-2020/project-4/data/data_8k


In [4]:
trainer = pl.Trainer(
    max_epochs=2,
    logger= loggers.TensorBoardLogger(config.LOG_DIR, name="M5"),
    gpus=1 if torch.cuda.is_available() else None,
    log_gpu_memory='all'
)

trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)


Loading cached train data from /nfs/students/summer-term-2020/project-4/data/data_8k
Loading cached val data from /nfs/students/summer-term-2020/project-4/data/data_8k


Set SLURM handle signals.

   | Name           | Type         | Params
--------------------------------------------
0  | model          | M5           | 555 K 
1  | model.model    | Sequential   | 555 K 
2  | model.model.0  | Conv1d       | 10 K  
3  | model.model.1  | BatchNorm1d  | 256   
4  | model.model.2  | MaxPool1d    | 0     
5  | model.model.3  | Dropout      | 0     
6  | model.model.4  | Conv1d       | 49 K  
7  | model.model.5  | BatchNorm1d  | 256   
8  | model.model.6  | MaxPool1d    | 0     
9  | model.model.7  | Dropout      | 0     
10 | model.model.8  | Conv1d       | 98 K  
11 | model.model.9  | BatchNorm1d  | 512   
12 | model.model.10 | MaxPool1d    | 0     
13 | model.model.11 | Dropout      | 0     
14 | model.model.12 | Conv1d       | 393 K 
15 | model.model.13 | BatchNorm1d  | 1 K   
16 | model.model.14 | MaxPool1d    | 0     
17 | model.model.15 | AvgPool1d    | 0     
18 | model.model.16 | PermuteLayer | 0     
19 | model.model.17 | Linear       | 1 K   
/nfs

Val-Acc=0.03793716656787196


/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Val-Acc=0.7919383521043272


Val-Acc=0.8168346176644932



1

In [ ]:
best_model = model

In [ ]:
def load_module(module_path, module_class):
    loaded_dict = torch.load(module_path)
    module = module_class(loaded_dict["hparams"])
    module.model.load_state_dict(loaded_dict["state_dict"])
    module.prepare_data()
    return module

model = load_module("jo1", M5PLModule)

In [6]:
#best_model = load_module("jo2", M5PLModule)
model.cuda()
model.report()

Accuracy: 	0.83
Precision: 	0.86
Recall: 	0.78
F1-Score: 	0.82

VAL-ACC: 1396/1687 (82.75044457617072%)

P-Rate: 	0.45


{'tp': array(643),
 'fp': array(108),
 'tn': array(753),
 'fn': array(183),
 'correct': array(1396),
 'n': 1687,
 'acc': 0.8275044457617071,
 'prec': 0.8561917443408789,
 'rec': 0.7784503631961259,
 'f1': 0.8154724159797084,
 'attack_args': None,
 'p_rate': 0.4451689389448725}

### Run a single attack

In [52]:
from attacks.pgd import ProjectedGradientDescent
attack = ProjectedGradientDescent(model.model, model.train_dataloader(), {"norm":"inf", "epsilon": .01}, early_stopping=-1, device='cuda', save_samples=True)
attack.attack()
attack.report( )

100%|██████████| 80/80 [00:11<00:00,  6.96it/s]

Attack-Parameters:	{'norm': 'inf', 'epsilon': 0.01}
Early stopping: 	False (-1)

Successfully attacked:	477
Total attacked: 	4155
Total processed:	5060

Success-Rate: 		0.11
Perturbed Accurracy: 	0.73


{'success_rate': 0.1148014440433213, 'acc': 0.7268774703557312}

In [53]:
print(len(attack.adversarial_examples))

477


In [42]:
#show_audio(attack.adversarial_examples[0])

adv_sample_1 = attack.adversarial_examples[1235]
adv_sample_1_data = adv_sample_1[2]
adv_sample_1_audio = adv_sample_1_data["audio"]
show_audio(adv_sample_1_audio)

print(adv_sample_1)

(tensor(0), tensor(1), {'audio': tensor([-0.2728, -0.2756, -0.3048,  ..., -0.0583, -0.0583, -0.0583]), 'sample_rate': tensor(8000)}, {'audio': tensor([ 1.0000,  1.0000,  1.0000,  ..., -0.0583, -0.0583, -0.0583]), 'sample_rate': tensor(8000)})


In [61]:
from attacks.pgd import ProjectedGradientDescent
from attacks.volume import VolumeAttack
#attack = ProjectedGradientDescent(model.model, model.train_dataloader(), {"norm":"inf", "epsilon": .01}, early_stopping=-1, device='cuda', save_samples=True)
attack = VolumeAttack(model.model, model.train_dataloader(), {}, early_stopping=-1, device='cuda', save_samples=True)


attack.attack()
attack.report( )
attack.showAdversarialExample(target_class=1)

  0%|          | 0/80 [00:00<?, ?it/s]


Exception: Not vectorized - TODO

* change:
    * return accuracy
    * make easy to compare via spectogram & waveform
